In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/city_data.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,haibara-akanedai,34.5333,135.9500,31.81,52,3,3.08,JP,1691718838
1,saint-philippe,-21.3585,55.7679,19.77,78,1,2.15,RE,1691718685
2,waitangi,-43.9535,-176.5597,12.12,89,100,3.13,NZ,1691718644
3,whitehorse,60.7161,-135.0538,20.05,76,75,1.03,CA,1691718631
4,ilulissat,69.2167,-51.1000,7.01,81,75,1.54,GL,1691718651


In [9]:
%%capture --no-display
# Configure the map plot
latitude = city_data_df["Lat"].iloc[0]
longitude = city_data_df["Lng"].iloc[0]
m = folium.Map(location=[latitude, longitude], zoom_start=2)

# Add markers for each city with the size of the point based on humidity
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row["Lat"], row["Lng"]],
        radius=row["Humidity"] / 5, 
        color="blue",
        fill=True,
        fill_color="blue",
        popup=f"City: {row['City']}<br>Humidity: {row['Humidity']}%",
    ).add_to(m)

# Display the map
m.save("humidity_map.html")  
display(m)

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 30) &
    (city_data_df["Humidity"] < 50)
].copy()  # Make a copy of the filtered DataFrame to avoid the warning

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
sample_data = filtered_cities_df.sample(10)  # Display a sample of 10 rows
sample_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
421,birjand,32.8663,59.2211,21.26,24,0,0.00,IR,1691718942
206,illizi,26.4833,8.4667,25.89,16,0,1.68,DZ,1691718889
185,port hedland,-20.3167,118.5667,29.03,21,1,3.09,AU,1691718884
462,as sulayyil,20.4607,45.5779,29.14,12,2,1.73,SA,1691718952
148,pirenopolis,-15.8525,-48.9592,24.27,44,0,2.42,BR,1691718875
147,south hedland,-20.4089,118.5983,28.96,21,1,3.09,AU,1691718875
257,sabha,27.0377,14.4283,25.72,32,0,4.60,LY,1691718900
439,bouarfa,32.5300,-1.9500,27.29,22,12,1.82,MA,1691718946
89,afif,23.9065,42.9172,26.73,14,19,2.27,SA,1691718861
71,vilhena,-12.7406,-60.1458,25.62,37,100,1.46,BR,1691718856


In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df 
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
sample_data = hotel_df.sample(10)  # Display a sample of 10 rows
sample_data

,City,Country,Lat,Lng,Humidity,Hotel Name
333,alice springs,AU,-23.7000,133.8833,20,
439,bouarfa,MA,32.5300,-1.9500,22,
148,pirenopolis,BR,-15.8525,-48.9592,44,
430,trinidad,BO,-14.8333,-64.9000,44,
149,ain oussera,DZ,35.4514,2.9058,29,
440,plettenberg bay,ZA,-34.0527,23.3716,35,
339,sao vicente,BR,-23.9631,-46.3919,26,
275,bukama,CD,-9.2000,25.8500,38,
89,afif,SA,23.9065,42.9172,14,
298,istgah-e rah ahan-e garmsar,IR,35.2346,52.3094,25,


In [12]:
# Set a larger radius for hotel search
radius = 10000

# Set other parameters for the Geoapify API request
params = {
    "filter": "category=hotels",
    "limit": 1,
    "apiKey": "geoapify_key"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"category=hotels&near={lat},{lng}&radius={radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
sample_data = hotel_df.sample(10)  # Display a sample of 10 rows
sample_data

Starting hotel search
mwene-ditu - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
saynshand - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
vilhena - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
afif - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
south hedland - nearest hotel: No hotel found
pirenopolis - nearest hotel: No hotel found
ain oussera - nearest hotel: No hotel found
port hedland - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
charters towers - nearest hotel: No hotel found
illizi - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
danghara - nearest hotel: No hotel found
darab - nearest hotel: No hotel found
taco pozo - nearest hotel: No hotel found
cuencame de ceniceros - nearest hotel: No hotel found
sabha - nearest hotel: No hotel found
deschutes river woods - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
430,trinidad,BO,-14.8333,-64.9000,44,No hotel found
194,petropavlovsk-kamchatsky,RU,53.0452,158.6483,49,No hotel found
147,south hedland,AU,-20.4089,118.5983,21,No hotel found
339,sao vicente,BR,-23.9631,-46.3919,26,No hotel found
315,kabul,AF,34.5000,69.4167,31,No hotel found
298,istgah-e rah ahan-e garmsar,IR,35.2346,52.3094,25,No hotel found
185,port hedland,AU,-20.3167,118.5667,21,No hotel found
206,illizi,DZ,26.4833,8.4667,16,No hotel found
106,karratha,AU,-20.7377,116.8463,35,No hotel found
439,bouarfa,MA,32.5300,-1.9500,22,No hotel found


In [ ]:
# Create sample data for demonstration
cities = ['City A', 'City B', 'City C']
humidity = [75, 60, 40]

# Configure the map plot
plt.figure(figsize=(10, 6))
plt.scatter(range(len(cities)), humidity, color='blue')
plt.title('City Humidity')
plt.xlabel('Cities')
plt.ylabel('Humidity')
plt.xticks(range(len(cities)), cities, rotation=45)

# Display the map
plt.tight_layout()

# Save the plot to a file (optional)
plt.savefig('humidity_map.png')

# Close the plot to prevent direct output
plt.close()